### COMP 473 Group Project

#### Facial Expression Recognition: 6-Class Classification

##### Import Libraries


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import pathlib

#### Training Metrics Graph
Automatically update loss and accuracy of training and validation set and display the graph as the model gets trained

In [ ]:
from IPython.display import clear_output

class PlotTrain(tf.keras.callbacks.Callback):

    def on_train_begin(self,logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []

    def on_epoch_end(self,epoch,logs={}):
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]

        #To plot the graph
        metrics = [x for x in logs if 'val' not in x]

        f, axs = plt.subplots(1, len(metrics), figsize = (15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), self.metrics[metric], label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), self.metrics['val_' + metric], label = 'val_'+metric)
        
        axs[i].legend()
        axs[i].grid()
        plt.tight_layout()
        plt.show()

##### Dataset Import

In [ ]:
def datasetImport(path):
    
    data_dir = pathlib.Path('D:\Concordia\COMP473\COMP_473_Project\CK+')
    image_count = len(list(data_dir.glob('*/*.png')))
    print("total image:", image_count)

    list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
    list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

    #Split between training and test set
    test_size = int(image_count * 0.3)
    train = list_ds.skip(test_size)
    test_ds = list_ds.take(test_size)

    #Split within training: training set and validation set
    val_size = int(len(train)*0.2)
    train_ds = train.skip(val_size)
    val_ds = train.take(val_size)

    return train_ds, val_ds, test_ds


In [ ]:
"""
    Replace 'path' with your directory's path
"""

# get training and validation data on dataset 1
train_ds1 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    # our original dataset is 48 pixels by 48 pixels
    image_size = (48,48),
    batch_size = 20 
)

test_ds1 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    image_size = (48,48), 
    batch_size = 20
)

# get training and validation data on dataset 2
train_ds2 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    image_size = (48,48),
    batch_size = 20 
)

test_ds2 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    image_size = (48,48), 
    batch_size = 20
)

# get training and validation data on dataset 3
train_ds3 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    image_size = (48,48),
    batch_size = 20 
)

test_ds3 = tf.keras.preprocessing.image_dataset_from_directory(
    'path', 
    image_size = (48,48), 
    batch_size = 20
)

##### Basic Model
No Pre-Processing Applied

<br>Model1_t1: Trained with Dataset 1

In [ ]:
model1_t1 = tf.keras.Sequential([

    # pooling layer (Downsampling)
    # downsize 2,2 pixels to 1,1
    # change image shape = (32, 32, 3)
    tf.keras.layers.MaxPooling2D((2,2)),

    #input_shape = shape of the images made by Ryan
    #kernel size change to 5x5
    tf.keras.layers.Conv2D( 32, (5,5), padding='none' , activation='relu', input_shape=(64,64,3)), 
    
    #After Conv2D image shape = (28, 28, 3)
    # pooling layer to change image shape = (14, 14, 3)
    tf.keras.layers.MaxPooling2D((2,2)),

    #kernel size change to 7x7
    tf.keras.layers.Conv2D(64, (7,7), padding='none' , activation='relu'), 
    
    #After Conv2D image shape = (8, 8, 3)
    # pooling layer to change image shape = (4, 4, 3)
    tf.keras.layers.MaxPooling2D((2,2)),
    
    # Flatten-Dense
    tf.keras.layers.Flatten(), 
    
    ## 1st layer
    # number inside Dense is the number of nodes. usually use 2^n
    tf.keras.layers.Dense(256, activation="relu"), 
    
    # helps prevent overfittng
    tf.keras.layers.Dropout(0.2),

    ## second layer. we must have 6 final nodes because we have 6 classes.
    # softmax: compress resulting number between 0~1, used in category problems. 
    # If you add up the probability of each class we get 1. 
    tf.keras.layers.Dense(6, activation="softmax") 
])

model1_t1.summary()

Compile and Fit(train) the basic models

<br> Model1_t1

In [ ]:
# compile and fit the model1_t1
callback_list = [PlotTrain()]
model1_t1.compile(loss="sparse_categorical_crossentropy", optimizer="adam" , metrics=['accuracy'])
model1_t1.fit(train_ds1, validation_data=val_ds1, epochs=30, callbacks=callback_list)
model1_t1.save("model1_t1.h5")

Classification Test and Results
<li> Classification Report
<li> Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
classes =['anger', 'contempt', 'disgust','fear','happy', 'sadness','surprise']

In [ ]:
model = tf.keras.Sequential([

    tf.keras.layers.Conv2D( 32, (3,3), padding='same' , activation='relu', input_shape=(48,48,3)), 
    
    # pooling layer (Downsampling)
    # downsize 2,2 pixels to 1,1
    tf.keras.layers.MaxPooling2D((2,2)),

    # Flatten-Dense
    tf.keras.layers.Flatten(), 
    
    ## 1st layer
    # number inside Dense is the number of nodes. usually use 2^n
    tf.keras.layers.Dense(64, activation="relu"), 
    
    # helps prevent overfittng
    tf.keras.layers.Dropout(0.2),

    ## second layer. we must have 7 final nodes because we have 7 classes.
    # softmax: compress resulting number between 0~1, used in category problems. 
    # If you add up the probability of each class we get 1. 
    tf.keras.layers.Dense(7, activation="softmax") 
])

Use model.load() to test with each model

<br>Model1_t1: Basic model with dataset #1

In [ ]:
# test_dir = 'pathToTestData'

#Fetch testing data from the dataset 1
test_dir1 = 'D:\Concordia\COMP473\COMP_473_Project\TestSet1-new'
test_generator1 = tf.keras.preprocessing.image.ImageDataGenerator()
test_ds1 = test_generator1.flow_from_directory(test_dir1, target_size=(48,48), batch_size=20, shuffle = False)

#storing actual class associated with each input image with the correct string label
y_true1 = test_ds1.classes
true_class1 = []
for i in y_true1:
    if(i==0):
        true_class1+=['anger']
    if(i==1):
        true_class1+=['contempt']
    if(i==2):
        true_class1+=['disgust']
    if(i==3):
        true_class1+=['fear']
    if(i==4):
        true_class1+=['happy']
    if(i==5):
        true_class1+=['sadness']
    if(i==6):
        true_class1+=['surprise']

#predict the category with the input images from the test set
model.load_weights('model1_t1.h5')
y_predict1= model.predict(test_ds1)
classes_x1 = np.argmax(y_predict1, axis=1)

#change the class labels to string
predicted_class1 = []
for i in classes_x1:
    if(i==0):
        predicted_class1+=['anger']
    if(i==1):
        predicted_class1+=['contempt']
    if(i==2):
        predicted_class1+=['disgust']
    if(i==3):
        predicted_class1+=['fear']
    if(i==4):
        predicted_class1+=['happy']
    if(i==5):
        predicted_class1+=['sadness']
    if(i==6):
        predicted_class1+=['surprise']



print(classification_report(true_class1, predicted_class1))
cf_model1 = confusion_matrix(true_class1, predicted_class1, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cf_model1, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.show()